In [4]:
import cv2
import albumentations as A
import numpy as np
from pathlib import Path

def read_yolo_label(label_path, image_width, image_height):
    """
    Read a YOLO format label file and convert bounding boxes to Albumentations format.
    YOLO format: [class_id, x_center, y_center, width, height] (normalized)
    Albumentations format: [x_min, y_min, x_max, y_max] (unnormalized)
    """
    if not label_path.exists():
        raise FileNotFoundError(f"Label file not found at {label_path}. Check the file path.")

    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    boxes = []
    class_ids = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        
        # Convert YOLO format to Albumentations format
        x_min = (x_center - width / 2) * image_width
        y_min = (y_center - height / 2) * image_height
        x_max = (x_center + width / 2) * image_width
        y_max = (y_center + height / 2) * image_height
        
        boxes.append([x_min, y_min, x_max, y_max])
        class_ids.append(class_id)
    
    return boxes, class_ids

def write_yolo_label(label_path, boxes, class_ids, image_width, image_height):
    """
    Convert bounding boxes back to YOLO format and save to a label file.
    """
    with open(label_path, 'w') as file:
        for box, class_id in zip(boxes, class_ids):
            x_min, y_min, x_max, y_max = box
            
            # Convert Albumentations format to YOLO format
            x_center = ((x_min + x_max) / 2) / image_width
            y_center = ((y_min + y_max) / 2) / image_height
            width = (x_max - x_min) / image_width
            height = (y_max - y_min) / image_height
            
            file.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def apply_augmentation(image, boxes, class_ids, augmentation_pipeline):
    """
    Apply augmentations to the image and bounding boxes.
    """
    transformed = augmentation_pipeline(image=image, bboxes=boxes, class_labels=class_ids)
    transformed_image = transformed['image']
    transformed_boxes = transformed['bboxes']
    transformed_class_ids = transformed['class_labels']
    
    return transformed_image, transformed_boxes, transformed_class_ids

def main():
    # Define the augmentation pipeline
    augmentation_pipeline = A.Compose([
        A.Resize(640, 640),  # Resize to YOLOv8 input size
        A.HorizontalFlip(p=0.5),  # Random horizontal flip
        A.RandomBrightnessContrast(p=0.2),  # Random brightness/contrast adjustment
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),  # Shift, scale, rotate
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

    # Example image and label paths
    image_path = Path("C:/Users/SCII1/Desktop/coal_size detector/data/large_dest/large_size_001.jpg").resolve()
    label_path = Path("C:/Users/SCII1/Desktop/coal_size detector/data/annotated_labels_dest/large_size_001.txt").resolve()

    # Debug: Print the paths
    print(f"Image path: {image_path}")
    print(f"Label path: {label_path}")

    # Check if the files exist
    print(f"Image exists: {image_path.exists()}")
    print(f"Label exists: {label_path.exists()}")

    # Load the image
    image = cv2.imread(str(image_path))
    if image is None:
        raise FileNotFoundError(f"Unable to read image at {image_path}. Check the file path and integrity.")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image_height, image_width, _ = image.shape

    # Load and convert bounding boxes
    boxes, class_ids = read_yolo_label(label_path, image_width, image_height)

    # Apply augmentations
    augmented_image, augmented_boxes, augmented_class_ids = apply_augmentation(image, boxes, class_ids, augmentation_pipeline)

    # Save the augmented image and labels
    output_image_path = Path("C:/Users/SCII1/Desktop/coal_size detector/data/augmented/image1_aug.jpg").resolve()
    output_label_path = Path("C:/Users/SCII1/Desktop/coal_size detector/data/augmented/label1_aug.txt").resolve()

    # Ensure the augmented image is a NumPy array
    if not isinstance(augmented_image, np.ndarray):
        raise TypeError(f"Augmented image is not a NumPy array. Got type: {type(augmented_image)}")

    # Save the augmented image
    cv2.imwrite(str(output_image_path), cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
    write_yolo_label(output_label_path, augmented_boxes, augmented_class_ids, image_width, image_height)

    print(f"Augmented image saved to: {output_image_path}")
    print(f"Augmented label saved to: {output_label_path}")

if __name__ == "__main__":
    main()

Image path: C:\Users\SCII1\Desktop\coal_size detector\data\large_dest\large_size_001.jpg
Label path: C:\Users\SCII1\Desktop\coal_size detector\data\annotated_labels_dest\large_size_001.txt
Image exists: True
Label exists: True
Augmented image saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\image1_aug.jpg
Augmented label saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\label1_aug.txt


In [ ]:
import cv2
import albumentations as A
import numpy as np
from pathlib import Path

def read_yolo_label(label_path, image_width, image_height):
    """
    Read a YOLO format label file and convert bounding boxes to Albumentations format.
    YOLO format: [class_id, x_center, y_center, width, height] (normalized)
    Albumentations format: [x_min, y_min, x_max, y_max] (unnormalized)
    """
    if not label_path.exists():
        raise FileNotFoundError(f"Label file not found at {label_path}. Check the file path.")

    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    boxes = []
    class_ids = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        
        # Convert YOLO format to Albumentations format
        x_min = (x_center - width / 2) * image_width
        y_min = (y_center - height / 2) * image_height
        x_max = (x_center + width / 2) * image_width
        y_max = (y_center + height / 2) * image_height
        
        boxes.append([x_min, y_min, x_max, y_max])
        class_ids.append(class_id)
    
    return boxes, class_ids

def write_yolo_label(label_path, boxes, class_ids, image_width, image_height):
    """
    Convert bounding boxes back to YOLO format and save to a label file.
    """
    with open(label_path, 'w') as file:
        for box, class_id in zip(boxes, class_ids):
            x_min, y_min, x_max, y_max = box
            
            # Convert Albumentations format to YOLO format
            x_center = ((x_min + x_max) / 2) / image_width
            y_center = ((y_min + y_max) / 2) / image_height
            width = (x_max - x_min) / image_width
            height = (y_max - y_min) / image_height
            
            file.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def apply_augmentation(image, boxes, class_ids, augmentation_pipeline):
    """
    Apply augmentations to the image and bounding boxes.
    """
    transformed = augmentation_pipeline(image=image, bboxes=boxes, class_labels=class_ids)
    transformed_image = transformed['image']
    transformed_boxes = transformed['bboxes']
    transformed_class_ids = transformed['class_labels']
    
    return transformed_image, transformed_boxes, transformed_class_ids

def main():
    # Define the augmentation pipeline
    augmentation_pipeline = A.Compose([
        A.Resize(640, 640),  # Resize to YOLOv8 input size
        A.HorizontalFlip(p=0.5),  # Random horizontal flip
        A.RandomBrightnessContrast(p=0.2),  # Random brightness/contrast adjustment
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),  # Shift, scale, rotate
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

    # Define source and destination directories
    source_images_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/large_dest").resolve()
    source_labels_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/annotated_labels_dest").resolve()
    augmented_images_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/augmented/images").resolve()
    augmented_labels_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/augmented/labels").resolve()

    # Create augmented directories if they don't exist
    augmented_images_dir.mkdir(parents=True, exist_ok=True)
    augmented_labels_dir.mkdir(parents=True, exist_ok=True)

    # Get all image and label files
    image_files = sorted(source_images_dir.glob("*.jpg"))
    label_files = sorted(source_labels_dir.glob("*.txt"))

    # Ensure the number of images and labels match
    if len(image_files) != len(label_files):
        raise ValueError(f"Mismatch between number of images ({len(image_files)}) and labels ({len(label_files)})")

    # Process each image and label pair
    for idx, (image_path, label_path) in enumerate(zip(image_files, label_files)):
        # Debug: Print the paths
        print(f"Processing image: {image_path}")
        print(f"Processing label: {label_path}")

        # Load the image
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Warning: Unable to read image at {image_path}. Skipping...")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        image_height, image_width, _ = image.shape

        # Load and convert bounding boxes
        boxes, class_ids = read_yolo_label(label_path, image_width, image_height)

        # Apply augmentations
        augmented_image, augmented_boxes, augmented_class_ids = apply_augmentation(image, boxes, class_ids, augmentation_pipeline)

        # Save the augmented image and labels
        output_image_path = augmented_images_dir / f"aug_{idx + 1:03d}.jpg"
        output_label_path = augmented_labels_dir / f"aug_{idx + 1:03d}.txt"

        # Ensure the augmented image is a NumPy array
        if not isinstance(augmented_image, np.ndarray):
            print(f"Warning: Augmented image is not a NumPy array. Skipping {image_path}...")
            continue

        # Save the augmented image
        cv2.imwrite(str(output_image_path), cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
        write_yolo_label(output_label_path, augmented_boxes, augmented_class_ids, image_width, image_height)

        print(f"Augmented image saved to: {output_image_path}")
        print(f"Augmented label saved to: {output_label_path}")

if __name__ == "__main__":
    main()

In [5]:
import cv2
import albumentations as A
import numpy as np
from pathlib import Path

def read_yolo_label(label_path, image_width, image_height):
    """
    Read a YOLO format label file and convert bounding boxes to Albumentations format.
    YOLO format: [class_id, x_center, y_center, width, height] (normalized)
    Albumentations format: [x_min, y_min, x_max, y_max] (unnormalized)
    """
    if not label_path.exists():
        raise FileNotFoundError(f"Label file not found at {label_path}. Check the file path.")

    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    boxes = []
    class_ids = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        
        # Convert YOLO format to Albumentations format
        x_min = (x_center - width / 2) * image_width
        y_min = (y_center - height / 2) * image_height
        x_max = (x_center + width / 2) * image_width
        y_max = (y_center + height / 2) * image_height
        
        boxes.append([x_min, y_min, x_max, y_max])
        class_ids.append(class_id)
    
    return boxes, class_ids

def write_yolo_label(label_path, boxes, class_ids, image_width, image_height):
    """
    Convert bounding boxes back to YOLO format and save to a label file.
    """
    with open(label_path, 'w') as file:
        for box, class_id in zip(boxes, class_ids):
            x_min, y_min, x_max, y_max = box
            
            # Convert Albumentations format to YOLO format
            x_center = ((x_min + x_max) / 2) / image_width
            y_center = ((y_min + y_max) / 2) / image_height
            width = (x_max - x_min) / image_width
            height = (y_max - y_min) / image_height
            
            file.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def apply_augmentation(image, boxes, class_ids, augmentation_pipeline):
    """
    Apply augmentations to the image and bounding boxes.
    """
    transformed = augmentation_pipeline(image=image, bboxes=boxes, class_labels=class_ids)
    transformed_image = transformed['image']
    transformed_boxes = transformed['bboxes']
    transformed_class_ids = transformed['class_labels']
    
    return transformed_image, transformed_boxes, transformed_class_ids

def main():
    # Define the augmentation pipeline
    augmentation_pipeline = A.Compose([
        A.Resize(640, 640),  # Resize to YOLOv8 input size
        A.HorizontalFlip(p=0.5),  # Random horizontal flip
        A.Rotate(limit=15, p=0.5),  # Random rotation (±15°)
        A.RandomScale(scale_limit=0.2, p=0.5),  # Random scaling (±20%)
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),  # Brightness/contrast adjustment
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),  # Hue/saturation adjustment
        A.GaussNoise(var_limit=(10, 50), p=0.5),  # Add Gaussian noise
        A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.5),  # Cutout augmentation
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

    # Define source and destination directories
    source_images_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/large_dest").resolve()
    source_labels_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/annotated_labels_dest").resolve()
    augmented_images_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/augmented/images").resolve()
    augmented_labels_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/augmented/labels").resolve()

    # Create augmented directories if they don't exist
    augmented_images_dir.mkdir(parents=True, exist_ok=True)
    augmented_labels_dir.mkdir(parents=True, exist_ok=True)

    # Get all image and label files
    image_files = sorted(source_images_dir.glob("*.jpg"))
    label_files = sorted(source_labels_dir.glob("*.txt"))

    # Ensure the number of images and labels match
    if len(image_files) != len(label_files):
        raise ValueError(f"Mismatch between number of images ({len(image_files)}) and labels ({len(label_files)})")

    # Process each image and label pair
    for idx, (image_path, label_path) in enumerate(zip(image_files, label_files)):
        # Debug: Print the paths
        print(f"Processing image: {image_path}")
        print(f"Processing label: {label_path}")

        # Load the image
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Warning: Unable to read image at {image_path}. Skipping...")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        image_height, image_width, _ = image.shape

        # Load and convert bounding boxes
        boxes, class_ids = read_yolo_label(label_path, image_width, image_height)

        # Apply augmentations
        augmented_image, augmented_boxes, augmented_class_ids = apply_augmentation(image, boxes, class_ids, augmentation_pipeline)

        # Save the augmented image and labels
        output_image_path = augmented_images_dir / f"aug_{idx + 1:03d}.jpg"
        output_label_path = augmented_labels_dir / f"aug_{idx + 1:03d}.txt"

        # Ensure the augmented image is a NumPy array
        if not isinstance(augmented_image, np.ndarray):
            print(f"Warning: Augmented image is not a NumPy array. Skipping {image_path}...")
            continue

        # Save the augmented image
        cv2.imwrite(str(output_image_path), cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
        write_yolo_label(output_label_path, augmented_boxes, augmented_class_ids, image_width, image_height)

        print(f"Augmented image saved to: {output_image_path}")
        print(f"Augmented label saved to: {output_label_path}")

if __name__ == "__main__":
    main()

C:\Users\SCII1\AppData\Local\Temp\ipykernel_3664\1957507428.py:70: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10, 50), p=0.5),  # Add Gaussian noise
C:\Users\SCII1\AppData\Local\Temp\ipykernel_3664\1957507428.py:71: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.5),  # Cutout augmentation


Processing image: C:\Users\SCII1\Desktop\coal_size detector\data\large_dest\large_size_001.jpg
Processing label: C:\Users\SCII1\Desktop\coal_size detector\data\annotated_labels_dest\large_size_001.txt
Augmented image saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\images\aug_001.jpg
Augmented label saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\labels\aug_001.txt
Processing image: C:\Users\SCII1\Desktop\coal_size detector\data\large_dest\large_size_002.jpg
Processing label: C:\Users\SCII1\Desktop\coal_size detector\data\annotated_labels_dest\large_size_002.txt
Augmented image saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\images\aug_002.jpg
Augmented label saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\labels\aug_002.txt
Processing image: C:\Users\SCII1\Desktop\coal_size detector\data\large_dest\large_size_003.jpg
Processing label: C:\Users\SCII1\Desktop\coal_size detector\data\annotated_labels_dest\large_si

In [ ]:
import cv2
import albumentations as A
import numpy as np
from pathlib import Path

def read_yolo_label(label_path, image_width, image_height):
    """
    Read a YOLO format label file and convert bounding boxes to Albumentations format.
    YOLO format: [class_id, x_center, y_center, width, height] (normalized)
    Albumentations format: [x_min, y_min, x_max, y_max] (unnormalized)
    """
    if not label_path.exists():
        raise FileNotFoundError(f"Label file not found at {label_path}. Check the file path.")

    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    boxes = []
    class_ids = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        
        # Convert YOLO format to Albumentations format
        x_min = (x_center - width / 2) * image_width
        y_min = (y_center - height / 2) * image_height
        x_max = (x_center + width / 2) * image_width
        y_max = (y_center + height / 2) * image_height
        
        boxes.append([x_min, y_min, x_max, y_max])
        class_ids.append(class_id)
    
    return boxes, class_ids

def write_yolo_label(label_path, boxes, class_ids, image_width, image_height):
    """
    Convert bounding boxes back to YOLO format and save to a label file.
    """
    with open(label_path, 'w') as file:
        for box, class_id in zip(boxes, class_ids):
            x_min, y_min, x_max, y_max = box
            
            # Convert Albumentations format to YOLO format
            x_center = ((x_min + x_max) / 2) / image_width
            y_center = ((y_min + y_max) / 2) / image_height
            width = (x_max - x_min) / image_width
            height = (y_max - y_min) / image_height
            
            file.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def apply_augmentation(image, boxes, class_ids, augmentation_pipeline):
    """
    Apply augmentations to the image and bounding boxes.
    """
    transformed = augmentation_pipeline(image=image, bboxes=boxes, class_labels=class_ids)
    transformed_image = transformed['image']
    transformed_boxes = transformed['bboxes']
    transformed_class_ids = transformed['class_labels']
    
    return transformed_image, transformed_boxes, transformed_class_ids

def main():
    # Define individual augmentation pipelines
    augmentations = [
        A.Compose([
            A.Resize(640, 640),  # Resize to YOLOv8 input size
            A.HorizontalFlip(p=1.0),  # Always apply horizontal flip
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),
        
        A.Compose([
            A.Resize(640, 640),
            A.Rotate(limit=15, p=1.0),  # Always apply rotation
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),
        
        A.Compose([
            A.Resize(640, 640),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),  # Always adjust brightness/contrast
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),
        
        A.Compose([
            A.Resize(640, 640),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1.0),  # Always adjust hue/saturation
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),
        
        A.Compose([
            A.Resize(640, 640),
            A.GaussNoise(var_limit=(10, 50), p=1.0),  # Always add Gaussian noise
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),
    ]

    # Define source and destination directories
    source_images_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/large_dest").resolve()
    source_labels_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/annotated_labels_dest").resolve()
    augmented_images_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/augmented/augmented_image").resolve()
    augmented_labels_dir = Path("C:/Users/SCII1/Desktop/coal_size detector/data/augmented/augmented_label").resolve()

    # Create augmented directories if they don't exist
    augmented_images_dir.mkdir(parents=True, exist_ok=True)
    augmented_labels_dir.mkdir(parents=True, exist_ok=True)

    # Get all image and label files
    image_files = sorted(source_images_dir.glob("*.jpg"))
    label_files = sorted(source_labels_dir.glob("*.txt"))

    # Ensure the number of images and labels match
    if len(image_files) != len(label_files):
        raise ValueError(f"Mismatch between number of images ({len(image_files)}) and labels ({len(label_files)})")

    # Process each image and label pair
    for idx, (image_path, label_path) in enumerate(zip(image_files, label_files)):
        # Debug: Print the paths
        print(f"Processing image: {image_path}")
        print(f"Processing label: {label_path}")

        # Load the image
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Warning: Unable to read image at {image_path}. Skipping...")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        image_height, image_width, _ = image.shape

        # Load and convert bounding boxes
        boxes, class_ids = read_yolo_label(label_path, image_width, image_height)

        # Apply each augmentation independently
        for aug_idx, augmentation_pipeline in enumerate(augmentations):
            # Apply augmentation
            augmented_image, augmented_boxes, augmented_class_ids = apply_augmentation(image, boxes, class_ids, augmentation_pipeline)

            # Save the augmented image and labels
            output_image_path = augmented_images_dir / f"aug_{idx + 1:03d}_{aug_idx + 1:03d}.jpg"
            output_label_path = augmented_labels_dir / f"aug_{idx + 1:03d}_{aug_idx + 1:03d}.txt"

            # Ensure the augmented image is a NumPy array
            if not isinstance(augmented_image, np.ndarray):
                print(f"Warning: Augmented image is not a NumPy array. Skipping {image_path}...")
                continue

            # Save the augmented image
            cv2.imwrite(str(output_image_path), cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
            write_yolo_label(output_label_path, augmented_boxes, augmented_class_ids, image_width, image_height)

            print(f"Augmented image saved to: {output_image_path}")
            print(f"Augmented label saved to: {output_label_path}")

if __name__ == "__main__":
    main()

C:\Users\SCII1\AppData\Local\Temp\ipykernel_3664\348040091.py:86: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10, 50), p=1.0),  # Always add Gaussian noise


Processing image: C:\Users\SCII1\Desktop\coal_size detector\data\large_dest\large_size_001.jpg
Processing label: C:\Users\SCII1\Desktop\coal_size detector\data\annotated_labels_dest\large_size_001.txt
Augmented image saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\images\aug_001_001.jpg
Augmented label saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\labels\aug_001_001.txt
Augmented image saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\images\aug_001_002.jpg
Augmented label saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\labels\aug_001_002.txt
Augmented image saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\images\aug_001_003.jpg
Augmented label saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\labels\aug_001_003.txt
Augmented image saved to: C:\Users\SCII1\Desktop\coal_size detector\data\augmented\images\aug_001_004.jpg
Augmented label saved to: C:\Users\SCII1\Desktop\coal_siz

In [13]:
aug_images = Path("C:/Users/SCII1/Desktop/coal_size detector/data/augmented/images")
image_list = aug_images.glob("*.jpg")

In [14]:
len(list(image_list))

1800